In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# ✅ Dataset Paths
train_dir = r"D:\DL project files\Deep learning\train"
val_dir = r"D:\DL project files\Deep learning\val"
test_dir = r"D:\DL project files\Deep learning\test"

In [3]:
# ✅ Image Augmentation & Rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)




In [4]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

# ✅ Data Loaderss
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,  # Keep this tunable
    class_mode='binary',
    shuffle=True  # ✅ Enables shuffling every epoch
)


val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)



Found 60000 images belonging to 2 classes.
Found 12858 images belonging to 2 classes.
Found 12858 images belonging to 2 classes.


In [5]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),  # More filters for complex features
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])


C:\Users\Pavan N A\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
#compile model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

optimizer = Adam(learning_rate=0.001)  # Start with 0.001
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
# ✅ Model Summary
model.summary()




Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    16,777,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,168,065 (65.49 MB)

 Trainable params: 17,167,105 (65.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train Model with Early Stopping
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[early_stopping]  # Include the callback here
)

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6945s 4s/step - accuracy: 0.5696 - loss: 0.6739 - val_accuracy: 0.6420 - val_loss: 0.6391
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6443s 3s/step - accuracy: 0.5891 - loss: 0.6649 - val_accuracy: 0.6454 - val_loss: 0.6175
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5610s 3s/step - accuracy: 0.6562 - loss: 0.6195 - val_accuracy: 0.7107 - val_loss: 0.5648
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5564s 3s/step - accuracy: 0.7036 - loss: 0.5792 - val_accuracy: 0.7740 - val_loss: 0.4835
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6520s 3s/step - accuracy: 0.7407 - loss: 0.5254 - val_accuracy: 0.7416 - val_loss: 0.7146
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5816s 3s/step - accuracy: 0.7720 - loss: 0.4809 - val_accuracy: 0.8158 - val_loss: 0.4380
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5655s 3s/step - accuracy: 0.7998 - loss: 0.4380 - val_accuracy: 0.6096 - val_loss: 0.9413
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5577s 3s/step - accuracy: 0.8192 -

In [13]:
# ✅ Evaluate on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# ✅ Save Model
model.save("deepfake_cnn_model.h5")
print("Model saved as deepfake_cnn_model.h5")

402/402 ━━━━━━━━━━━━━━━━━━━━ 333s 829ms/step - accuracy: 0.8183 - loss: 0.3620


Test Accuracy: 0.8858
Model saved as deepfake_cnn_model.h5


In [14]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# ✅ Load the saved model
model = load_model("deepfake_cnn_model.h5")

# ✅ Load and preprocess the test image
img_path = r"C:\Users\Pavan N A\Desktop\sem 6\DL\Ass-01\data\Test_10k\Real_Test\00368.jpg" # Change this to your test image path
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize if your model was trained with normalization

# ✅ Predict
prediction = model.predict(img_array)[0][0]

prediction = 1 - prediction

# ✅ Print the results
if prediction >= 0.5:
    print(f"✅ Prediction: FAKE with {(prediction * 100):.2f}% confidence")
else:
    print(f"✅ Prediction: REAL with {((1 - prediction) * 100):.2f}% confidence")
    


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
✅ Prediction: REAL with 95.57% confidence
